# Il problema del Number Partitioning

TODO: sarebbe carino aggiungere una breve spiegazione del problema.

1. Definisco la classe che rappresenta il problema

In [1]:
import numpy as np

np.random.seed(22)   # set random seed

class Problem:
    def __init__(self, num):
        self.dim = num
        self.numbers = np.random.randint(1, 100000, num)

    def objective_function(self, sol):
        s = 0
        for i in range(self.dim):
            if sol[i] == 0:
                s += self.numbers[i]
            else:
                s -= self.numbers[i]
                
        return np.abs(s)
    
    def get_dim(self):
        return self.dim

In [2]:
problem = Problem(100)
initial_sol = np.array(np.random.randint(0, 1+1, problem.get_dim()))

print(initial_sol)

[0 1 1 1 1 0 0 1 0 1 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 1 1 0 0 1 0 1 0 1 1 0 1
 0 0 1 0 0 1 1 1 1 0 0 1 0 0 0 0 1 1 1 1 1 1 1 0 0 1 0 0 0 0 1 0 0 0 1 1 1
 0 1 1 0 1 0 0 1 1 0 1 1 0 0 0 1 1 0 1 0 1 0 0 1 1 1]


In [3]:
def local_search(prob, init_sol=None, verbose=False):
    n  = prob.get_dim()
    
    x  = np.random.randint(0, 1+1, n) if init_sol is None else init_sol.copy()
    fx = prob.objective_function(x)
    
    
    if verbose: print(f"initial value {fx}")
    
    improved = True
    while improved:
        best_f = 1e300
        
        for i in range(n):
            x[i] = 1 - x[i]
            fy = prob.objective_function(x)
            
            if fy < best_f:
                y = x.copy()
                best_f = fy

            x[i] = 1 - x[i]
            
        if best_f < fx:
            fx = best_f
            x  = y
            improved = True
            if verbose: print(f"new value {fx}")

        else:
            improved = False
            
    return x,fx




In [4]:
x, fx = local_search(problem, initial_sol, verbose=True)

print(x, fx)

initial value 297224
new value 99972
new value 1924
new value 114
[0 1 1 1 1 0 0 1 0 1 0 1 0 1 1 1 0 0 0 0 1 0 0 0 0 1 1 0 0 1 0 1 0 1 1 0 1
 0 0 1 0 0 1 1 1 1 1 0 1 0 0 0 0 1 1 1 1 1 1 1 0 0 1 0 0 0 0 1 0 0 0 1 1 1
 0 1 1 0 1 0 0 1 1 0 1 1 0 0 0 1 1 0 1 0 1 0 0 1 1 1] 114


In [5]:
def local_search_first_impr(prob, init_sol=None, verbose=False):
    n = prob.get_dim()
    
    x  = np.random.randint(0, 1+1, n) if init_sol is None else init_sol.copy()
    fx = prob.objective_function(x)
    
    if verbose: print(f"initial value {fx}")

    improved=True
    while improved:
        best_f = fx
        ordering = list(range(n))
        np.random.shuffle(ordering)
        for i in ordering:
            x[i] = 1 - x[i]
            fy = prob.objective_function(x)
            
            if fy < best_f:
                y = x.copy()
                best_f = fy
                x[i] = 1 - x[i]
                break
                
            x[i] = 1 - x[i]
            
        if best_f < fx:
            fx = best_f
            x = y
            improved=True
            if verbose: print(f"new value {fx}")
        else:
            improved=False
    
    return x,fx

In [6]:
x_impr, fx_impr = local_search_first_impr(problem, initial_sol, verbose=True)

print(x_impr, fx_impr)

initial value 297224
new value 248156
new value 218538
new value 31542
new value 24646
new value 1172
new value 866
new value 70
[0 1 1 1 1 0 0 1 0 1 1 1 0 0 1 1 0 0 0 0 1 0 0 0 0 1 1 0 0 1 0 1 0 0 1 0 1
 0 0 1 0 0 1 1 1 1 0 0 0 0 0 0 0 1 1 1 1 1 1 1 0 0 1 0 0 0 0 1 0 1 0 1 1 1
 0 1 1 0 1 1 0 1 1 0 1 1 0 0 0 1 1 0 1 1 1 0 0 1 1 1] 70


In [7]:
print("Local Search vs First Impr")
# print("x == x_impr") if (x == x_impr).all() else print("x != x_impr")
print(x == x_impr)
print(f"ls_fx: {fx}", f"fi_fx: {fx_impr}")

Local Search vs First Impr
[ True  True  True  True  True  True  True  True  True  True False  True
  True False  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True False  True  True
  True  True  True  True  True  True  True  True  True  True False  True
 False  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True False  True  True
  True  True  True  True  True  True  True False  True  True  True  True
  True  True  True  True  True  True  True  True  True False  True  True
  True  True  True  True]
ls_fx: 114 fi_fx: 70


## Iterated Local Search

In [8]:
def iterated_local_search(prob, num_tries, num_flips, init_sol=None):
    def perturbation(x, num_flips):
        n = len(x)
        y = x.copy()
        
        for flip in range(num_flips):
            i = np.random.randint(0, n)
            y[i] = 1 - y[i]
        
        return y

    n  = prob.get_dim()
    x  = np.random.randint(0, 1+1, n) if init_sol is None else init_sol.copy()
    fx = prob.objective_function(x)

    nt = 0
    while nt < num_tries:
        y    = perturbation(x, num_flips)
        z,fz = local_search(prob, y)
        
        if fz < fx:
            x  = z
            fx = fz
            nt = 0
        else:
            nt += 1

    return x,fx